In [2]:
import requests
import json

In [59]:
base_url = 'https://es.xxxxx.com/recipes'
headers = {'accept': 'application/json'}

def get_recipes_page(page=None):
    payload = {'page': page}
    r = requests.get(f'{base_url}?page={page}', headers=headers)
    if r.status_code != 200:
        print(r.status_code)
        raise AssertionError
    return r

In [60]:
paging_data = get_recipes_page().json()['data']['pagingData']
paging_data

{'prelink': '/',
 'current': 1,
 'previous': None,
 'next': 2,
 'first': None,
 'last': 369,
 'range': [1, 2, 3, 4, 5],
 'fromResult': 1,
 'toResult': 9,
 'totalResult': 3314,
 'pageCount': 369}

In [71]:
recipes_data = []

In [72]:
for i in range(1, paging_data['pageCount'] + 1):
    print(f'Page {i}...')
    req = get_recipes_page(i)
    page_recipes = [r['fields'] for r in req.json()['data']['mixedPosts']]
    recipes_data.extend(page_recipes)

Page 1...
Page 2...
Page 3...
Page 4...
Page 5...
Page 6...
Page 7...
Page 8...
Page 9...
Page 10...
Page 11...
Page 12...
Page 13...
Page 14...
Page 15...
Page 16...
Page 17...
Page 18...
Page 19...
Page 20...
Page 21...
Page 22...
Page 23...
Page 24...
Page 25...
Page 26...
Page 27...
Page 28...
Page 29...
Page 30...
Page 31...
Page 32...
Page 33...
Page 34...
Page 35...
Page 36...
Page 37...
Page 38...
Page 39...
Page 40...
Page 41...
Page 42...
Page 43...
Page 44...
Page 45...
Page 46...
Page 47...
Page 48...
Page 49...
Page 50...
Page 51...
Page 52...
Page 53...
Page 54...
Page 55...
Page 56...
Page 57...
Page 58...
Page 59...
Page 60...
Page 61...
Page 62...
Page 63...
Page 64...
Page 65...
Page 66...
Page 67...
Page 68...
Page 69...
Page 70...
Page 71...
Page 72...
Page 73...
Page 74...
Page 75...
Page 76...
Page 77...
Page 78...
Page 79...
Page 80...
Page 81...
Page 82...
Page 83...
Page 84...
Page 85...
Page 86...
Page 87...
Page 88...
Page 89...
Page 90...
Page 91...
Page 92.

In [ ]:
recipes_data

In [73]:
len(recipes_data)

3314

#  

In [2]:
import pickle
import datetime

In [ ]:
with open(f'xxxxx_{datetime.date.today()}.pkl', 'wb') as f:
    pickle.dump(recipes_data, f)

#  

In [3]:
import pickle
import datetime

In [4]:
file_date = '2020-06-19'

In [5]:
with open(f'xxxxx_{file_date if file_date else datetime.date.today()}.pkl', 'rb') as f:
    dishes_data = pickle.load(f)

In [6]:
row_keys = set()
for r in dishes_data:
    for k in r.keys():
        row_keys.add(k)
row_keys

{'__origPaywallUntilDate',
 'accentColor',
 'amagiData',
 'attributionText',
 'author',
 'availableForVod',
 'description',
 'episodeNumber',
 'feedTitle',
 'hasRecipes',
 'isBranded',
 'isLinear',
 'isLive',
 'isTMTVLive',
 'landscapeAsset',
 'landscapeAssetClean',
 'landscapeHeroImage',
 'landscapeThumbnail',
 'landscapeThumbnailClean',
 'linearReady',
 'paywallUntilDate',
 'portraitAsset',
 'portraitAssetClean',
 'portraitHeroImage',
 'portraitThumbnail',
 'portraitThumbnailClean',
 'premium',
 'primaryColor',
 'publishDate',
 'recipeJSON',
 'seasonNumber',
 'sendOOLandscapeToAmagi',
 'slug',
 'squareAsset',
 'squareThumbnail',
 'tags',
 'title',
 'verticals',
 'vimeoPublishMeta'}

### Armo el dataset de recetas

In [8]:
len(dishes_data)

3314

In [10]:
# Filtro aquellos que no tienen receta o imagen
dishes_data = [
    r for r in dishes_data
    if ('hasRecipes' in r.keys() and r['hasRecipes'] and len(r['recipeJSON']) > 0
        and 'landscapeThumbnail' in r.keys())
              ]
len(dishes_data)

2676

In [11]:
# recipe_keys = set()
count = 0
for d in dishes_data:
    for r in d['recipeJSON']:
        if 'servings' not in r.keys():
            # recipe_keys.add(d)
            count += 1
            break
count

594

In [1]:
#[(r['title'], [recipe['name'] for recipe in r['recipeJSON']]) for r in dishes_data if len(r['recipeJSON']) > 1]

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


Algunos platos tienen más de una receta.

### Creo el dataframe

In [14]:
dishes_list = [
    {
        'title': d['title'],
        'description': d.get('description', ''),
        'recipes': [{
            'name': r['name'],
            'steps': r['steps'],
            'prepTime': r.get('prepTime'),
            'activeTime': r.get('activeTime'),
            'cookTime': r.get('cookTime'),
            'totalTime': r.get('totalTime'),
            'servings': r.get('servings'),
            'ingredients': r['ingredients'],
            'source': 'Tastemade',
        } for r in d['recipeJSON']],
        'tags': d.get('tags', []),
        'slug': d['slug'],
        'assets': {
            'thumbnail': d['landscapeThumbnail']['fields']['file']['url'],
            'video': d['landscapeAsset']['fields']['awsOriginal'] if d.get('landscapeAsset') else None,
        }
    } for d in dishes_data
]
print(len(dishes_list))
dishes_list[0]

2676


{'title': 'Barras de Mantequilla de Maní y Caramelo',
 'description': 'Adicción asegurada! 🤤',
 'recipes': [{'name': 'Barras de mantequilla de maní y caramelo ',
   'steps': ['Corteza',
    'Para hacer la corteza, agregue las galletitas de chocolate a un procesador y mezcle hasta que se hagan migajas. Detenga la procesadora y agregue la mantequilla derretida. Mezcle para unir las migajas con la mantequilla derretida.',
    'Vierte las migajas húmedas en un molde para pastel cuadrado de 23x23 cm forrado con papel de hornear y usa la parte posterior de una cuchara para untar uniformemente y presionar las migajas en el fondo del molde. Enfriar durante 30 minutos en la heladera.',
    '',
    'Relleno de mantequilla de maní',
    'Agregue todos los ingredientes a un tazón grande y colóquelo en el microondas durante 20 segundos a la vez, revolviendo cada vez hasta que quede suave. Verter en el molde para hornear y extender uniformemente. Coloque en la heladera durante 20 minutos.',
    '',


In [15]:
import pandas as pd

In [16]:
dishes_df = pd.DataFrame(dishes_list)
dishes_df

,title,description,recipes,tags,slug,assets
0,Barras de Mantequilla de Maní y Caramelo,Adicción asegurada! 🤤,[{'name': 'Barras de mantequilla de maní y car...,[],barras-de-mantequilla-de-mani-y-caramelo,{'thumbnail': '//truffle-assets.imgix.net/3edf...
1,Pastel de Carne con Queso,"También llamado Shakshuka, este Pastel de Carn...","[{'name': 'pastel de carne con queso', 'steps'...",[],pastel-de-carne-con-queso,{'thumbnail': '//truffle-assets.imgix.net/a3f2...
2,Wraps de Espagueti y Carne,,"[{'name': 'Wraps de espagueti y carne', 'steps...",[],wraps-de-espagueti-y-carne,{'thumbnail': '//truffle-assets.imgix.net/74d0...
3,Pastel de Abejas,El panal de Abejas más hermoso que vas a ver h...,"[{'name': 'pastel de abejas', 'steps': ['Hacer...",[],pastel-de-abejas,{'thumbnail': '//truffle-assets.imgix.net/535f...
4,Galletas de Fresa,El premio a la mejor imitación! 🏆,"[{'name': 'galletas de fresa', 'steps': ['Haga...",[],galletas-de-fresa,{'thumbnail': '//truffle-assets.imgix.net/50b8...
...,...,...,...,...,...,...
2671,Manzana con Palomitas,,"[{'name': 'Manzana con Palomitas', 'steps': ['...","[postres, niños, merienda, dulces]",manzana-con-palomitas,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2672,ChocoBrownie,Tus dos postres preferidos en uno. ¡No lo podr...,"[{'name': 'ChocoBrownie', 'steps': ['Derretir ...","[dulces, postres, brownies, chocolate, dulce d...",chocobrownie,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2673,Cheesecake de Oreo y Fresas sin Cocción,Cuando tu galletita preferida se junta con una...,[{'name': 'Cheesecake de Frutilla sin Cocción'...,"[tortas, dulces, merienda, cheesecake, oreo, f...",cheesecake-de-frutilla-sin-coccion,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2674,Torta Crocante de Banana y Chocolate,,[{'name': 'Torta Crocante de Banana y Chocolat...,"[tortas, dulces, postres, chocolate, banana, h...",torta-crocante-de-banana-y-chocolate,{'thumbnail': '//truffle-assets.imgix.net/1t1b...


In [17]:
dishes_df.iloc[9].recipes

[{'name': '4 recetas con limonada',
  'steps': ['limonada de arándanos',
   'Prepare el jarabe de arándano agregando los arándanos, el azúcar, el agua y el jugo de limón en una sartén. Continúe revolviendo y cocine durante 10-15 minutos hasta que espese.',
   'Combine 1/2 taza de jarabe de arándanos con la limonada y el agua. Mezclar hasta que esté bien combinado.',
   'Agregue la cantidad deseada de jarabe de arándano al fondo de una taza, luego agregue la mezcla de limonada en el vaso.',
   'Cubra con una bola de su helado favorito. ¡Disfrutar!',
   '',
   'limonada de mango y coco',
   'Vierta la lata de leche de coco en una bandeja de cubitos de hielo y congele hasta que esté duro.',
   'Coloque los cubitos de hielo congelados de leche de coco en una licuadora con la limonada y el jarabe simple. Licúa hasta que esté completamente combinado.',
   'Para el jarabe de jengibre, agregue azúcar, agua y rodajas de jengibre en una sartén para saltear, y cocine por 10 minutos. Ponga a un la

In [18]:
dishes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2676 non-null   object
 1   description  2676 non-null   object
 2   recipes      2676 non-null   object
 3   tags         2676 non-null   object
 4   slug         2676 non-null   object
 5   assets       2676 non-null   object
dtypes: object(6)
memory usage: 125.6+ KB


In [19]:
dishes_df.describe()

,title,description,recipes,tags,slug,assets
count,2676,2676,2676,2676,2676,2676
unique,2628,2459,2674,1114,2676,2676
top,Pollo Frito,,[{'name': 'Cheesecake Vegana de Granada y Fram...,[],pasta-facil-en-1-bowl,{'thumbnail': '//truffle-assets.imgix.net/60f9...
freq,3,157,2,1558,1,1


### Proceso un poco el dataframe

In [20]:
# Busco títulos repetidos
dishes_df[dishes_df.duplicated(['title'])].sort_values(['title'])

,title,description,recipes,tags,slug,assets
2432,Brownies de Chocolate Blanco,Nunca creíste que podías hacerlos blancos.,"[{'name': 'Brownies de Chocolate Blanco', 'ste...","[merienda, chocolate, horno, postres, dulces, ...",brownies-de-chocolate-blanco,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2484,Budín de Dulce de Leche,Budín goloso.,"[{'name': 'Budín de Dulce de Leche', 'steps': ...","[merienda, horno, dulce de leche, postres, dul...",budin-de-dulce-de-leche,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2518,Carrot Cake,"Torta de Zanahoria, suena increible!","[{'name': 'Carrot Cake', 'steps': ['Poner en u...","[dulces, queso crema, horno, postres, merienda...",carrot-cake,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2428,Cazuela de Pollo,¡Sin vueltas!,"[{'name': 'Cazuela de Pollo', 'steps': ['Corta...","[papa, almuerzos, saladas, rápidas, fáciles, v...",cazuela-de-pollo,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2345,Cerdo Agridulce,Carré de Cerdo con Salsa Agridulce,"[{'name': 'Cerdo Agridulce', 'steps': ['Para e...","[salsas, fritas, cenas, almuerzos, carne de ce...",cerdo-agridulce,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2138,Cheesecake de Dulce de Leche,¡Amamos el dulce de leche!,"[{'name': 'Cheesecake de Dulce de Leche', 'ste...","[cheesecake, dulce de leche, tortas, postres, ...",cheesecake-de-dulce-de-leche,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2045,Cheesecake de Frutos Rojos,¡Puro y exclusivo sabor!,"[{'name': 'Cheesecake de Frutos Rojos', 'steps...","[tortas, cheesecake, postres, dulces, queso cr...",cheesecake-de-frutos-rojos,{'thumbnail': '//truffle-assets.imgix.net/8ef1...
2265,Chorizos a la Pomarola,Lleva tus chorizos a otro nivel con esta salsa...,"[{'name': 'Chorizos a la Pomarola', 'steps': [...","[salsas, chorizo, cenas, tomate, saladas, verd...",chorizos-a-la-pomarola,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2675,Croque Monsieur,Sandwich de Queso y Lomito,"[{'name': 'Croque Monsieur', 'steps': ['Realiz...","[saladas, rápidas, fáciles, horno, merienda, a...",croque-monsieur-1,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2083,Croquetas de Papa,,"[{'name': 'Croquetas de Papa', 'steps': ['En u...","[papa, saladas, bocadillos, panceta, queso, pa...",croquetas-de-papa,{'thumbnail': '//truffle-assets.imgix.net/1t1b...


**To do:** hay platos repetidos o por muy similares.

In [26]:
from collections import defaultdict

def get_tag_counts(df):
    tag_counts = defaultdict(int)
    for i, r in df.iterrows():
        for tag in r.tags:
            tag_counts[tag] += 1
    return tag_counts

In [32]:
tag_counts = get_tag_counts(dishes_df)
print(f'Different tags: {len(tag_counts)}')
print(f'Total count: {sum(tag_counts.values())}')
tag_counts

Different tags: 357
Total count: 8044


defaultdict(int,
            {'Frutas': 23,
             'Dulces': 159,
             'Fáciles': 168,
             'Rápidas': 127,
             'Postres': 45,
             'Postres Frios': 17,
             'Frutillas': 13,
             'Limón': 9,
             'Merienda': 67,
             'Queso': 70,
             'Fritas': 31,
             'Saladas': 194,
             'Carne': 41,
             'Carne de Cerdo': 2,
             'Almuerzos': 64,
             'Pastas': 23,
             'Fideos': 22,
             'Invierno': 23,
             'Salsas': 4,
             'Vegetarianas': 27,
             'Chocolate': 73,
             'Niños': 56,
             'Menos de 5 ingredientes': 3,
             'Tortas': 46,
             'Postres frios': 1,
             'brownie': 1,
             'Verduras': 35,
             'Saludables': 42,
             'Palta': 9,
             'Panadería': 3,
             'Pan': 17,
             'Papa': 31,
             'Sandwiches': 3,
             'Pasteles': 20,
  

In [29]:
a = ' Lim ón  '
a.lower().strip()

'lim ón'

In [54]:
# for i, r in dishes_df.iterrows():
#     dishes_df.

def clean_up_tag(tag):
    return tag.lower().strip()

def print_stuff(tl):
    print(tl)

dishes_df['tags'] = dishes_df['tags'].apply(
    lambda tl: [clean_up_tag(t) for t in tl]
)

In [55]:
# Veamos de nuevo los tags
tag_counts = get_tag_counts(dishes_df)
print(f'Different tags: {len(tag_counts)}')
print(f'Total count: {sum(tag_counts.values())}')
tag_counts

Different tags: 221
Total count: 8044


defaultdict(int,
            {'frutas': 39,
             'dulces': 430,
             'fáciles': 383,
             'rápidas': 304,
             'postres': 270,
             'postres frios': 66,
             'frutillas': 38,
             'limón': 42,
             'merienda': 206,
             'queso': 278,
             'fritas': 94,
             'saladas': 542,
             'carne': 140,
             'carne de cerdo': 26,
             'almuerzos': 239,
             'pastas': 77,
             'fideos': 47,
             'invierno': 32,
             'salsas': 27,
             'vegetarianas': 57,
             'chocolate': 263,
             'niños': 88,
             'menos de 5 ingredientes': 95,
             'tortas': 147,
             'brownie': 3,
             'verduras': 146,
             'saludables': 47,
             'palta': 33,
             'panadería': 3,
             'pan': 59,
             'papa': 77,
             'sandwiches': 3,
             'pasteles': 37,
             'horno': 

Redujo un 50 % la cantidad de tags distintos.

In [88]:
# Cantidad de rows sin tags
dishes_df['tags'].value_counts()[0:1]

[]    1558
Name: tags, dtype: int64

In [113]:
for k in ['activeTime', 'cookTime', 'prepTime', 'totalTime']:
    for rr in dishes_df['recipes']:
        for r in rr:
            if not (r[k] == '' or type(r[k]) in [int, type(None)]):
                print(r[k])

5
45
125
35
10
60
20
30
30
10 min
20
25
30
20
1 hora
35
25
9
20
30
20
15
40
30
25
30
30
60
30
40
40
80
20
10
20
20
20
40
35
80
60
20
60
80
80
100
30
90
10
10
80
45
40
80
20
40
40
40
30
120
45
8
20
40
6
30
60
20
60
45
60
60
60
40
35
20
40
3-6
5
8-10
35
45
60
30
40
60
45
40
540
80
50
65
20
30
20
40
15
1
60
120
30
40
30
35
45
25
30
25
20
40
210
30
50
25
15
15
20
20
30
30
80
8-12
25
15
40
30
35
20
25
40
15
30
40
35
15
30
25
30
35
30
25
30
60
45
40
30
40
25
35
15
35
30
25
15
30
5
5
60
15
20
190
50
60
30
30
15
65
10
15
20
20
30
20
140
25
70
30
45
25
25
20
4
240
150
65
25
90
25
60
2h 30
2 horas
25
35
2h 30
1h 40
5 horas
25
50
25
40
35
30
60
30
150
120
30
40
30
440
15
30
90
40
60
60
45
30
10
45
45
40
20
60
15
30
90
60
60
50
50
310
10
10
60
20
60
60
30
40
30
40
30


**To do:** normalizar los tiempos. Algunos aclaran la unidad, no siempre en el mismo formato, algunos son strings y otros muchos son int.

Tal vez aplanar todo a `int` (y después ver si no siempre mostrarlo en minutos). Hay un par de casos más difíciles: rangos.

In [139]:
# Busco rangos
for k in ['activeTime', 'cookTime', 'prepTime', 'totalTime']:
    for rr in dishes_df['recipes']:
        for r in rr:
            if type(r[k]) not in [int, type(None)] and '-' in r[k]:
                # print(r)
                print(r[k])

3-6
8-10
8-12


Solo hay 3 casos de rangos de tiempo, por lo que podría corregirlos a mano para un valor razonable.

In [162]:
dishes_df2 = dishes_df.copy()

casos:
* es None -> manener
* int -> mantener
* string vacío -> None
* string con letras -> sacarlas
* string con número -> intentar convertir
  * Xh Ym -> 60X + Y
  * 

In [160]:
for i, d in dishes_df.iterrows():
    for r in d.recipes:
        for k in ['activeTime', 'cookTime', 'prepTime', 'totalTime']:
            # Casos especiales
            if r[k] == '3-6':
                r[k] = 5
            if r[k] in ['8-10', '8-12']:
                r[k] = 10
            if r[k] == '10 min':
                r[k] = 10
            if r[k] == '1 hora':
                r[k] = 60
            if r[k] == '2h 30':
                r[k] = 150
            if r[k] == '2 horas':
                r[k] = 120
            if r[k] == '1h 40':
                r[k] = 100
            if r[k] == '5 horas':
                r[k] = 300
            
            if r[k] == '':
                r[k] = None
            if type(r[k]) in [type(None), int]:
                 continue
            r[k] = int(r[k])

In [161]:
for k in ['activeTime', 'cookTime', 'prepTime', 'totalTime']:
    for rr in dishes_df['recipes']:
        for r in rr:
            if not (r[k] == '' or type(r[k]) in [int, type(None)]):
                print(r[k])
# nada nadines

In [165]:
# Veamos los servings
count = 0
for k in ['servings']:
    for rr in dishes_df['recipes']:
        for r in rr:
            if r[k] == '':
                count += 1
            if not (r[k] == '' or type(r[k]) in [int, type(None)]):
                print(r[k])
print()
print(count)

6
2
24 copas
10
4-6
4
12
10 bocaditos
12
4 Mini Tacos
12
10 Bocaditos
8 Bocaditos
4 Mini tacos
6
16 cookies
6
13
6
4
2
170
4
4
10
8
8
4
1
9
4
6
6
6
8
8
4
4
6
4
4
4
4
8
5
6
8
8
4
9
6
8
2
2
4
4
6
4
5
2
6
4
4
3
8
6
20
4
4
6
4
1
1
3
4
8
4
4
6-8
3
3
4-5
8-12
10-15
4
4-6
4
6-8
4
4
3
4
12 a 15
4
5-6
3-4
4
2
4
14
1
2
12
3
8
2
4
4
10
8
6
4
2
4
3-4
8
8
4-6
8
3-4
3
4
3
8

4201


In [167]:
# Ahora servings
for i, d in dishes_df.iterrows():
    for r in d.recipes:
        if r[k] == '':
            r[k] = None

In [168]:
count = 0
for k in ['servings']:
    for rr in dishes_df['recipes']:
        for r in rr:
            if r[k] == '':
                count += 1
            if not (r[k] == '' or type(r[k]) in [int, type(None)]):
                print(r[k])
print()
print(count)

6
2
24 copas
10
4-6
4
12
10 bocaditos
12
4 Mini Tacos
12
10 Bocaditos
8 Bocaditos
4 Mini tacos
6
16 cookies
6
13
6
4
2
170
4
4
10
8
8
4
1
9
4
6
6
6
8
8
4
4
6
4
4
4
4
8
5
6
8
8
4
9
6
8
2
2
4
4
6
4
5
2
6
4
4
3
8
6
20
4
4
6
4
1
1
3
4
8
4
4
6-8
3
3
4-5
8-12
10-15
4
4-6
4
6-8
4
4
3
4
12 a 15
4
5-6
3-4
4
2
4
14
1
2
12
3
8
2
4
4
10
8
6
4
2
4
3-4
8
8
4-6
8
3-4
3
4
3
8

0


In [129]:
c = pd.DataFrame([
    {
        'a1': [{'a': None}, {'a': '2'}],
        'b1': 8,
    }, {
        'a1': [{'a': None}, {'a': '2'}],
        'b1': 0,
    }
])

In [136]:
e = 0
for i, d in c.iterrows():
    for r in d.a1:
        r['a'] = e
        e += 1
c

,a1,b1
0,"[{'a': 0}, {'a': 1}]",8
1,"[{'a': 2}, {'a': 3}]",0


In [169]:
dishes_df

,title,description,recipes,tags,slug,assets
0,Barras de Mantequilla de Maní y Caramelo,Adicción asegurada! 🤤,[{'name': 'Barras de mantequilla de maní y car...,[],barras-de-mantequilla-de-mani-y-caramelo,{'thumbnail': '//truffle-assets.imgix.net/3edf...
1,Pastel de Carne con Queso,"También llamado Shakshuka, este Pastel de Carn...","[{'name': 'pastel de carne con queso', 'steps'...",[],pastel-de-carne-con-queso,{'thumbnail': '//truffle-assets.imgix.net/a3f2...
2,Wraps de Espagueti y Carne,,"[{'name': 'Wraps de espagueti y carne', 'steps...",[],wraps-de-espagueti-y-carne,{'thumbnail': '//truffle-assets.imgix.net/74d0...
3,Pastel de Abejas,El panal de Abejas más hermoso que vas a ver h...,"[{'name': 'pastel de abejas', 'steps': ['Hacer...",[],pastel-de-abejas,{'thumbnail': '//truffle-assets.imgix.net/535f...
4,Galletas de Fresa,El premio a la mejor imitación! 🏆,"[{'name': 'galletas de fresa', 'steps': ['Haga...",[],galletas-de-fresa,{'thumbnail': '//truffle-assets.imgix.net/50b8...
...,...,...,...,...,...,...
2671,Manzana con Palomitas,,"[{'name': 'Manzana con Palomitas', 'steps': ['...","[postres, niños, merienda, dulces]",manzana-con-palomitas,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2672,ChocoBrownie,Tus dos postres preferidos en uno. ¡No lo podr...,"[{'name': 'ChocoBrownie', 'steps': ['Derretir ...","[dulces, postres, brownies, chocolate, dulce d...",chocobrownie,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2673,Cheesecake de Oreo y Fresas sin Cocción,Cuando tu galletita preferida se junta con una...,[{'name': 'Cheesecake de Frutilla sin Cocción'...,"[tortas, dulces, merienda, cheesecake, oreo, f...",cheesecake-de-frutilla-sin-coccion,{'thumbnail': '//truffle-assets.imgix.net/1t1b...
2674,Torta Crocante de Banana y Chocolate,,[{'name': 'Torta Crocante de Banana y Chocolat...,"[tortas, dulces, postres, chocolate, banana, h...",torta-crocante-de-banana-y-chocolate,{'thumbnail': '//truffle-assets.imgix.net/1t1b...


### Finalmente, guardar en CSV

In [171]:
dishes_df.to_csv('

# Más análisis

In [3]:
import pandas as pd

In [4]:
dishes_df = pd.read_csv('datasets/dishes_xxxxx.csv')

In [5]:
dishes_df[0:20]

,Unnamed: 0,title,description,recipes,tags,slug,assets
0,0,Barras de Mantequilla de Maní y Caramelo,Adicción asegurada! 🤤,[{'name': 'Barras de mantequilla de maní y car...,[],barras-de-mantequilla-de-mani-y-caramelo,{'thumbnail': '//truffle-assets.imgix.net/3edf...
1,1,Pastel de Carne con Queso,"También llamado Shakshuka, este Pastel de Carn...","[{'name': 'pastel de carne con queso', 'steps'...",[],pastel-de-carne-con-queso,{'thumbnail': '//truffle-assets.imgix.net/a3f2...
2,2,Wraps de Espagueti y Carne,NaN,"[{'name': 'Wraps de espagueti y carne', 'steps...",[],wraps-de-espagueti-y-carne,{'thumbnail': '//truffle-assets.imgix.net/74d0...
3,3,Pastel de Abejas,El panal de Abejas más hermoso que vas a ver h...,"[{'name': 'pastel de abejas', 'steps': ['Hacer...",[],pastel-de-abejas,{'thumbnail': '//truffle-assets.imgix.net/535f...
4,4,Galletas de Fresa,El premio a la mejor imitación! 🏆,"[{'name': 'galletas de fresa', 'steps': ['Haga...",[],galletas-de-fresa,{'thumbnail': '//truffle-assets.imgix.net/50b8...
5,5,Crème Brûlée de Piña,Ya sabes como darle un toque original a tu pos...,"[{'name': 'Creme Brûlée de piña', 'steps': ['P...",[],creme-brulee-de-pinaa,{'thumbnail': '//truffle-assets.imgix.net/35cb...
6,6,Caramelos Caseros,Quién va a rellenar su caramelera hoy? 🤩 🍬,"[{'name': 'Caramelo salado', 'steps': ['', 'Po...",[],caramelo-salado,{'thumbnail': '//truffle-assets.imgix.net/e71c...
7,7,Pastel Perruno,NaN,"[{'name': 'pastel perruno', 'steps': ['', 'pas...",[],pastel-perruno,{'thumbnail': '//truffle-assets.imgix.net/d0f2...
8,8,Postre Sorpresa de Algodón de Azúcar,"Las sorpresas siempre son lindas, y más sin so...",[{'name': 'Postre Sorpresa de Algodón de Azúca...,[],postre-sorpresa-de-algodon-de-azucar,{'thumbnail': '//truffle-assets.imgix.net/3b8b...
9,9,4 Recetas con Limonada,"Una misma base, distintos tragos deliciosos! 🍹","[{'name': '4 recetas con limonada', 'steps': [...",[],4-recetas-con-limonada,{'thumbnail': '//truffle-assets.imgix.net/5db4...


In [7]:
dishes_df.iloc[7].recipes

'[{\'name\': \'pastel perruno\', \'steps\': [\'\', \'pastel\', \'Precaliente un horno forzado por ventilador a 160C (320F) o 180C (356F) para un horno convencional. Rocíe una lata de pastel de 8 "que tiene 4 pulgadas de profundidad. Alinear con papel de hornear. Dejar de lado.\', \'En el tazón de una batidora de pie equipada con el accesorio de paleta, agregue el cacao en polvo, la harina, la sal, el bicarbonato de sodio y el azúcar. ¡Encienda la batidora a baja velocidad y permita que se mezcle durante un par de minutos para ayudar a que todo se combine bien! (alternativamente, puede hacer esto tamizando los ingredientes juntos). Agregue la mantequilla ablandada, no hay mantequilla visible y está bien incorporada en el seco.\', \'Luego, agregue leche y huevos a los ingredientes secos y mezcle hasta que estén bien combinados y suaves.\', \'Raspe el tazón y mezcle durante otros 20 segundos.\', \'Vierta la masa en el molde para pastel preparado previamente y hornee por 45 minutos o hasta